In [54]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import shutil
import numpy as np
import tensorflow as tf
import seaborn as sns
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
from platform import python_version

print(python_version())

import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.applications import DenseNet121,InceptionV3
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
# Specific neural network models & layer types
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

3.10.5


In [55]:
# Check for TensorFlow GPU access
print(f"TensorFlow has access to the following devices:\n{tf.config.list_physical_devices()}")

# See TensorFlow version
print(f"TensorFlow version: {tf.__version__}")


TensorFlow has access to the following devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow version: 2.16.2


In [56]:
# Path to Images Dataset Images
Images_path_part1 = '/Users/darylwanji/Desktop/Computer_Vision_Capstone_Project/data/HAM10000_images_part_1'
Images_path_part2 = '/Users/darylwanji/Desktop/Computer_Vision_Capstone_Project/data/HAM10000_images_part_2'

# Path to All Images in Dataset
final_path = '/Users/darylwanji/Desktop/Computer_Vision_Capstone_Project/data/All_Images'

In [57]:

def copy_image_files(src_folder, dest_folder):
    """
    Copy image files from the source folder to the destination folder.

    Parameters:
    src_folder (str): Path to the source folder containing image files.
    dest_folder (str): Path to the destination folder where image files will be copied.

    Returns:
    None
    """
    for filename in os.listdir(src_folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            shutil.copy(os.path.join(src_folder, filename), dest_folder)

        
# Create the destination folder if it doesn't exist
if not os.path.exists(final_path):
    os.makedirs(final_path)

copy_image_files(Images_path_part1, final_path)
copy_image_files(Images_path_part2, final_path)

In [58]:
meta_data_df = pd.read_csv('/Users/darylwanji/Desktop/Computer_Vision_Capstone_Project/data/Updated_metadata.csv')
meta_data_df.sample(5)

,lesion_id,image_id,Diagnosis,Confirmation_Method,Age,Sex,Location,Image_path
2392,HAM_0006748,ISIC_0027210,vasc,consensus,30,female,lower extremity,/Users/darylwanji/Desktop/Computer_Vision_Caps...
4340,HAM_0004298,ISIC_0025648,nv,follow_up,40,male,upper extremity,/Users/darylwanji/Desktop/Computer_Vision_Caps...
5644,HAM_0003861,ISIC_0026951,nv,follow_up,40,female,back,/Users/darylwanji/Desktop/Computer_Vision_Caps...
7937,HAM_0004043,ISIC_0033792,nv,histo,65,female,back,/Users/darylwanji/Desktop/Computer_Vision_Caps...
9833,HAM_0002461,ISIC_0025953,akiec,histo,65,male,face,/Users/darylwanji/Desktop/Computer_Vision_Caps...


In [59]:
# Add Images' Path to Metadata
meta_data_df['Image_path'] = meta_data_df['image_id'].apply(lambda x: os.path.join(final_path, f"{x}.jpg"))
#Save Meta Data to new file
meta_data_df.to_csv('/Users/darylwanji/Desktop/Computer_Vision_Capstone_Project/data/Updated_metadata.csv',index=False)

In [60]:
meta_data_df

,lesion_id,image_id,Diagnosis,Confirmation_Method,Age,Sex,Location,Image_path
0,HAM_0000118,ISIC_0027419,bkl,histo,80,male,scalp,/Users/darylwanji/Desktop/Computer_Vision_Caps...
1,HAM_0000118,ISIC_0025030,bkl,histo,80,male,scalp,/Users/darylwanji/Desktop/Computer_Vision_Caps...
2,HAM_0002730,ISIC_0026769,bkl,histo,80,male,scalp,/Users/darylwanji/Desktop/Computer_Vision_Caps...
3,HAM_0002730,ISIC_0025661,bkl,histo,80,male,scalp,/Users/darylwanji/Desktop/Computer_Vision_Caps...
4,HAM_0001466,ISIC_0031633,bkl,histo,75,male,ear,/Users/darylwanji/Desktop/Computer_Vision_Caps...
...,...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40,male,abdomen,/Users/darylwanji/Desktop/Computer_Vision_Caps...
10011,HAM_0002867,ISIC_0033550,akiec,histo,40,male,abdomen,/Users/darylwanji/Desktop/Computer_Vision_Caps...
10012,HAM_0002867,ISIC_0033536,akiec,histo,40,male,abdomen,/Users/darylwanji/Desktop/Computer_Vision_Caps...
10013,HAM_0000239,ISIC_0032854,akiec,histo,80,male,face,/Users/darylwanji/Desktop/Computer_Vision_Caps...


In [61]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit LabelEncoder and transform 'Region_Of_Disease' column
meta_data_df['Updated_Diagnosis_Label'] = label_encoder.fit_transform(meta_data_df['Diagnosis'])

# Display the mapping between original labels and encoded labels
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Encoding Mapping:")
print(label_mapping)

Label Encoding Mapping:
{'akiec': 0, 'bcc': 1, 'bkl': 2, 'df': 3, 'mel': 4, 'nv': 5, 'vasc': 6}


In [62]:
meta_data_df.sample(5)

,lesion_id,image_id,Diagnosis,Confirmation_Method,Age,Sex,Location,Image_path,Updated_Diagnosis_Label
339,HAM_0002988,ISIC_0025337,bkl,histo,35,male,face,/Users/darylwanji/Desktop/Computer_Vision_Caps...,2
8,HAM_0005132,ISIC_0025837,bkl,histo,70,female,back,/Users/darylwanji/Desktop/Computer_Vision_Caps...,2
6393,HAM_0005002,ISIC_0031869,nv,follow_up,35,male,back,/Users/darylwanji/Desktop/Computer_Vision_Caps...,5
8384,HAM_0005625,ISIC_0031623,nv,histo,35,female,back,/Users/darylwanji/Desktop/Computer_Vision_Caps...,5
3595,HAM_0006385,ISIC_0026733,nv,follow_up,50,female,trunk,/Users/darylwanji/Desktop/Computer_Vision_Caps...,5


# Base Model 
This Model will be done without working on the Class Imbalance

In [63]:
# Shuffle Dataset 
meta_data_df = meta_data_df.sample(frac=1, random_state=42).reset_index(drop=True)
meta_data_df

,lesion_id,image_id,Diagnosis,Confirmation_Method,Age,Sex,Location,Image_path,Updated_Diagnosis_Label
0,HAM_0007180,ISIC_0033272,mel,histo,65,male,face,/Users/darylwanji/Desktop/Computer_Vision_Caps...,4
1,HAM_0007195,ISIC_0031923,nv,histo,40,female,lower extremity,/Users/darylwanji/Desktop/Computer_Vision_Caps...,5
2,HAM_0001835,ISIC_0026652,mel,histo,65,male,back,/Users/darylwanji/Desktop/Computer_Vision_Caps...,4
3,HAM_0000465,ISIC_0030583,bkl,consensus,35,female,trunk,/Users/darylwanji/Desktop/Computer_Vision_Caps...,2
4,HAM_0001720,ISIC_0034010,nv,histo,45,male,abdomen,/Users/darylwanji/Desktop/Computer_Vision_Caps...,5
...,...,...,...,...,...,...,...,...,...
10010,HAM_0005460,ISIC_0028199,nv,follow_up,50,female,lower extremity,/Users/darylwanji/Desktop/Computer_Vision_Caps...,5
10011,HAM_0005615,ISIC_0031067,nv,follow_up,45,female,trunk,/Users/darylwanji/Desktop/Computer_Vision_Caps...,5
10012,HAM_0005773,ISIC_0031096,nv,follow_up,45,female,lower extremity,/Users/darylwanji/Desktop/Computer_Vision_Caps...,5
10013,HAM_0001053,ISIC_0030998,bkl,consensus,55,female,chest,/Users/darylwanji/Desktop/Computer_Vision_Caps...,2


In [ ]:

# Separate features (X) and target variable (y)
X = Sample_Meta_data_df.drop(columns=['Updated_Diagnosis_Label']) 
y = Sample_Meta_data_df['Updated_Diagnosis_Label']

# Display the shape of X and y
print("Shape of features (X):", X.shape)
print("Shape of target variable (y):", y.shape)

Shape of features (X): (500, 8)
Shape of target variable (y): (500,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### CNN



In [ ]:
import random

def set_random_seed(seed):

    '''Random seeds for reproducability'''

    random.seed(seed)
    tf.random.set_seed(seed)

In [ ]:
CNN_model = Sequential()

# Create simple CNN model architecture with Pooling for dimensionality reduction
# and Dropout to reduce overfitting
CNN_model.add(Conv2D(32, # number of filters/kernels
                     kernel_size=(3, 3), # size of each kernel
                     activation = 'relu', # same as before
                     input_shape = (28, 28, 1))) # 1st layer, say how to connect
CNN_model.add(MaxPooling2D(pool_size=(2, 2)))
CNN_model.add(Dropout(0.25)) # helpful for CNN overfitting

CNN_model.add(Conv2D(64, (3, 3), activation='relu')) # another 64 3,3 filters
CNN_model.add(MaxPooling2D(pool_size=(2, 2))) # pooling 2x2
CNN_model.add(Dropout(0.25))

# Flatten the output of our convolutional layers
CNN_model.add(Flatten())

# Add dense layers
# This is the classifier part
CNN_model.add(Dense(128, activation='relu'))
CNN_model.add(Dense(64, activation='relu'))
CNN_model.add(Dense(7, activation='softmax'))

# Print out a summary of the network
CNN_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 232,455 (908.03 KB)

 Trainable params: 232,455 (908.03 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile the model with the desired loss function, optimizer, and metric(s) to track
CNN_model.compile(loss = 'sparse_categorical_crossentropy',
                  optimizer = 'Adam',
                  metrics = ['accuracy'])

In [ ]:
# Fit the model on the training data, defining desired batch_size & number of epochs,
# running validation after each batch
# THIS WILL TAKE A LONG TIME TO RUN!!!
CNN_model.fit(X_train, y_train,
              batch_size = 128,
              epochs = 10,
              verbose = 1,
              validation_split = 0.2)